KFold Cross Validation Python Tutorial
https://github.com/codebasics/py/blob/master/ML/12_KFold_Cross_Validation/12_k_fold.ipynb

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import svm, datasets

#import the data
iris = datasets.load_iris()

In [8]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['rbf','linear']
}, cv=10, return_train_score=False)
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00099237, 0.00049992, 0.00059831, 0.00040083, 0.0004992 ,
        0.00080249]),
 'std_fit_time': array([0.00077286, 0.00049993, 0.00048852, 0.00049096, 0.00049925,
        0.00040151]),
 'mean_score_time': array([0.00050321, 0.00010009, 0.00040016, 0.00019968, 0.00039918,
        0.00019937]),
 'std_score_time': array([0.00067397, 0.00030026, 0.00049011, 0.00039935, 0.0004889 ,
        0.00039873]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [11]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000992,0.000773,0.000503,0.000674,1,rbf,"{'C': 1, 'kernel': 'rbf'}",1.0,0.933333,1.0,1.0,1.000000,0.933333,0.933333,1.0,1.0,1.0,0.980000,0.030551,1
1,0.000500,0.000500,0.000100,0.000300,1,linear,"{'C': 1, 'kernel': 'linear'}",1.0,0.933333,1.0,1.0,0.866667,1.000000,0.933333,1.0,1.0,1.0,0.973333,0.044222,3
2,0.000598,0.000489,0.000400,0.000490,10,rbf,"{'C': 10, 'kernel': 'rbf'}",1.0,0.933333,1.0,1.0,0.866667,1.000000,0.933333,1.0,1.0,1.0,0.973333,0.044222,3
3,0.000401,0.000491,0.000200,0.000399,10,linear,"{'C': 10, 'kernel': 'linear'}",1.0,1.000000,1.0,1.0,0.866667,1.000000,0.933333,1.0,1.0,1.0,0.980000,0.042687,1
4,0.000499,0.000499,0.000399,0.000489,20,rbf,"{'C': 20, 'kernel': 'rbf'}",1.0,0.933333,1.0,1.0,0.866667,0.933333,0.933333,1.0,1.0,1.0,0.966667,0.044721,5
5,0.000802,0.000402,0.000199,0.000399,20,linear,"{'C': 20, 'kernel': 'linear'}",1.0,1.000000,1.0,1.0,0.866667,0.933333,0.866667,1.0,1.0,1.0,0.966667,0.053748,5


In [12]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [13]:
clf.best_score_

0.9800000000000001

In [15]:
#Random search to reduce computational cost
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=10, 
    return_train_score=False, 
    n_iter=5
)
rs.fit(iris.data, iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,20,linear,0.966667
1,10,rbf,0.973333
2,20,rbf,0.966667
3,1,linear,0.973333
4,10,linear,0.980000


How about different models with different hyperparameters?

In [16]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [17]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 1}
2,logistic_regression,0.966667,{'C': 5}
